In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, roc_auc_score

In [10]:
#load the data and split it into training and active player datasets
df = pd.read_csv('../data/master.csv')
df['is_active'] = df['end_year'] >= 2023
train_df = df[df['is_active'] == False]
active_df = df[df['is_active'] == True]

In [19]:
X = train_df.drop(columns=['hof', 'name', 'id',
                  'start_year', 'end_year', 'pos', 'is_active'])
y = train_df['hof']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [14]:
lr = LogisticRegression(class_weight='balanced', max_iter=1000) #we use a balanced data set to avoid biasing the model towards the majority class
lr.fit(X_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(class_weight='balanced', max_iter=1000)

In [15]:
rf = RandomForestClassifier(class_weight='balanced', random_state=42) # Random Forest model for classification
rf.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', random_state=42)

In [16]:
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss') #we use the logloss metric for evaluation
xgb.fit(X_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [17:03:16] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, ...)

In [17]:
models = {
    "Logistic Regression": lr,
    "Random Forest": rf,
    "XGBoost": xgb
}

for name, model in models.items():
    print(f"--- {name} ---")
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    print("Classification Report:")
    print(classification_report(y_test, y_pred))

    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

    print("ROC AUC Score:", roc_auc_score(y_test, y_prob))
    print("\n")

Logistic Regression:
               precision    recall  f1-score   support

           0       0.99      0.96      0.98      1463
           1       0.71      0.95      0.82       166

    accuracy                           0.96      1629
   macro avg       0.85      0.95      0.90      1629
weighted avg       0.97      0.96      0.96      1629

Random Forest:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      1463
           1       1.00      1.00      1.00       166

    accuracy                           1.00      1629
   macro avg       1.00      1.00      1.00      1629
weighted avg       1.00      1.00      1.00      1629

XGBoost:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      1463
           1       1.00      1.00      1.00       166

    accuracy                           1.00      1629
   macro avg       1.00      1.00      1.00      1629
weighted avg       1.00   

In [18]:
active_df['hof_prob'] = rf.predict_proba(X_active)[:, 1]
active_df[['name', 'hof_prob']].sort_values(
    by='hof_prob', ascending=False).head(10)

/var/folders/br/z7w8p8pd1sgccb6psg9x9d4m0000gn/T/ipykernel_28518/241098767.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  active_df['hof_prob'] = rf.predict_proba(X_active)[:, 1]


,name,hof_prob
514,Kevin Durant,0.98
54,Giannis Antetokounmpo,0.97
1878,Russell Westbrook,0.96
1071,Damian Lillard,0.93
414,Stephen Curry,0.92
1384,Chris Paul,0.92
884,LeBron James,0.92
727,James Harden,0.88
428,Anthony Davis,0.84
859,Kyrie Irving,0.78
